In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rubiks
import torch
import torch.nn as nn
import torch.nn.functional as F
device= 'cpu'
import rubiks
import rubiks2
import torch

# Models

In [2]:
class DQN(nn.Module):
    def __init__(self, num_inputs, hidden, num_actions, non_linearity):
        super(DQN, self).__init__()
        
        self.num_inputs = num_inputs
        self.hidden = hidden
        self.num_actions = num_actions
        self.non_linearity = non_linearity
        
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(num_inputs, self.hidden[0]))
        
        previous = self.hidden[0]
        for hidden_layer_size in self.hidden[1:]:
            self.layers.append(nn.Linear(previous, hidden_layer_size))
            previous = hidden_layer_size
            
        self.layers.append(nn.Linear(previous, num_actions))        
        
    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = self.non_linearity(self.layers[i](x))
        return self.layers[-1](x)
    
    def increase_capacity(self, increment):
        for i in range(len(self.hidden)):
            self.hidden[i] += increment[i]
        
        weight = self.layers[0].weight.data
        self.layers[0] = nn.Linear(self.num_inputs, self.hidden[0])
        if increment[0]>0:
            self.layers[0].weight.data[0:-increment[0],:] = weight
        else:
            self.layers[0].weight.data[0:,:] = weight
        
        for i in range(1, len(self.layers) - 1):
            weight = self.layers[i].weight.data
            self.layers[i] = nn.Linear(self.hidden[i-1], self.hidden[i])
            if increment[i] > 0:
                if increment[i-1] >0: 
                    self.layers[i].weight.data[0:-increment[i],0:-increment[i-1]] = weight
                else:
                    self.layers[i].weight.data[0:-increment[i],0:] = weight
            else:
                if increment[i-1] >0:
                    self.layers[i].weight.data[0:,0:-increment[i-1]] = weight
                else:
                    self.layers[i].weight.data[0:,0:] = weight
        
        weight = self.layers[-1].weight.data
        self.layers[-1] = nn.Linear(self.hidden[-1], self.num_actions)
        if increment[-1] >0:
            self.layers[-1].weight.data[:,0:-increment[-1]] = weight
        else:
            self.layers[-1].weight.data[:,0:] = weight
        
    def decrease_capacity(self, increment):
        for i in range(len(self.hidden)):
            self.hidden[i] -= increment[i]
        
        weight = self.layers[0].weight.data
        self.layers[0] = nn.Linear(self.num_inputs, self.hidden[0])
        self.layers[0].weight.data = weight[0:-increment[0],:]
        
        for i in range(1, len(self.layers) - 1):
            weight = self.layers[i].weight.data
            self.layers[i] = nn.Linear(self.hidden[i-1], self.hidden[i])
            self.layers[i].weight.data = weight[0:-increment[i],0:-increment[i-1]]
        
        weight = self.layers[-1].weight.data
        self.layers[-1] = nn.Linear(self.hidden[-1], self.num_actions)
        self.layers[-1].weight.data = weight[:,0:-increment[-1]]
    
    def act(self, state, epsilon, mask):
        if np.random.rand() > epsilon:
            state = torch.tensor([state], dtype=torch.float32, device=device)
            mask = torch.tensor([mask], dtype=torch.float32, device=device)
            q_values = self.forward(state) + mask
            action = q_values.max(1)[1].view(1, 1).item()
        else:
            action =  np.random.randint(self.num_actions)
        return action

In [3]:
class DuelingDQN(nn.Module):
    def __init__(self, num_inputs, hidden, num_actions, non_linearity):
        super(DuelingDQN, self).__init__()
        
        self.num_inputs = num_inputs
        self.hidden = hidden
        self.num_actions = num_actions
        
        self.non_linearity = non_linearity
        
        self.sharedLayers = nn.ModuleList()
        self.sharedLayers.append(nn.Linear(num_inputs, self.hidden[0]))
        
        previous = self.hidden[0]
        for hidden_layer_size in self.hidden[1:-1]:
            self.sharedLayers.append(nn.Linear(previous, hidden_layer_size))
            previous = hidden_layer_size

        self.adv1 = nn.Linear(previous, self.hidden[-1])
        self.adv2 = nn.Linear(self.hidden[-1], num_actions)
        
        self.v1 = nn.Linear(previous, self.hidden[-1])
        self.v2 = nn.Linear(self.hidden[-1], 1)
        
    def forward(self, x):
        for i in range(len(self.sharedLayers)):
            
            x = self.non_linearity(self.sharedLayers[i](x))
            
        a = self.non_linearity(self.adv1(x))
        a = self.adv2(a)
        
        v = self.non_linearity(self.v1(x))
        v = self.v2(v)
        
        return v + a - a.mean()
    
    def increase_capacity(self, increment):
        for i in range(len(self.hidden)):
            self.hidden[i] += increment[i]
        
        # Check whether the increment isn't zero
        if increment[0] > 0:
            weight = self.sharedLayers[0].weight.data
            self.sharedLayers[0] = nn.Linear(self.num_inputs, self.hidden[0])
            self.sharedLayers[0].weight.data[0:-increment[0],:] = weight

        for i in range(1, len(self.sharedLayers)):
            weight = self.sharedLayers[i].weight.data
            self.sharedLayers[i] = nn.Linear(self.hidden[i-1], self.hidden[i])
            if increment[i] > 0:
                if increment[i-1] > 0:
                    self.sharedLayers[i].weight.data[0:-increment[i],0:-increment[i-1]] = weight
                else:
                    self.sharedLayers[i].weight.data[0:-increment[i],0:] = weight
            else:
                if increment[i-1] > 0:
                        self.sharedLayers[i].weight.data[0:,0:-increment[i-1]] = weight
                else:
                    self.sharedLayers[i].weight.data[0:,0:] = weight
            
        weight_adv1 = self.adv1.weight.data
        self.adv1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        
        weight_v1 = self.v1.weight.data
        self.v1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        if increment[-1] > 0:
            if increment[-2] > 0:
                self.adv1.weight.data[0:-increment[-1],0:-increment[-2]] = weight_adv1
                self.v1.weight.data[0:-increment[-1],0:-increment[-2]] = weight_v1
            else:
                self.adv1.weight.data[0:-increment[-1],0:] = weight_adv1
                self.v1.weight.data[0:-increment[-1],0:] = weight_v1
        else:
            if increment[-2] > 0:
                self.adv1.weight.data[0:,0:-increment[-2]] = weight_adv1
                self.v1.weight.data[0:,0:-increment[-2]] = weight_v1
            else:
                self.adv1.weight.data[0:,0:] = weight_adv1
                self.v1.weight.data[0:,0:] = weight_v1
            
        weight_adv2 = self.adv2.weight.data
        self.adv2 = nn.Linear(self.hidden[-1], self.num_actions)
        
        weight_v2 = self.v2.weight.data
        self.v2 = nn.Linear(self.hidden[-1], 1)
        
        if increment[-1] > 0:
            self.adv2.weight.data[:,0:-increment[-1]] = weight_adv2
            self.v2.weight.data[:,0:-increment[-1]] = weight_v2
        else:
            self.adv2.weight.data[:,0:] = weight_adv2
            self.v2.weight.data[:,0:] = weight_v2
        
    def decrease_capacity(self, increment):
        for i in range(len(self.hidden)):
            self.hidden[i] -= increment[i]

        weight = self.sharedLayers[0].weight.data
        self.sharedLayers[0] = nn.Linear(self.num_inputs, self.hidden[0])
        self.sharedLayers[0].weight.data = weight[0:-increment[0],:]
        
        for i in range(1, len(self.sharedLayers)):
            weight = self.sharedLayers[i].weight.data
            self.sharedLayers[i] = nn.Linear(self.hidden[i-1], self.hidden[i])
            self.sharedLayers[i].weight.data = weight[0:-increment[i],0:-increment[i-1]]
            
        weight = self.adv1.weight.data
        self.adv1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        self.adv1.weight.data = weight[0:-increment[-1],0:-increment[-2]]
            
        weight = self.adv2.weight.data
        self.adv2 = nn.Linear(self.hidden[-1], self.num_actions)
        self.adv2.weight.data = weight[:,0:-increment[-1]]
        
        weight = self.v1.weight.data
        self.v1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        self.v1.weight.data = weight[0:-increment[-1],0:-increment[-2]]
            
        weight = self.v2.weight.data
        self.v2 = nn.Linear(self.hidden[-1], 1)
        self.v2.weight.data = weight[:,0:-increment[-1]]
        
    def act(self, state, epsilon, mask):
        if np.random.rand() > epsilon:
            state = torch.tensor([state], dtype=torch.float32, device=device)
            q_values = self.forward(state)
            action = q_values.max(1)[1].view(1, 1).item()
        else:
            action =  np.random.randint(self.num_actions)
        return action

In [4]:
class DuelingDQNHER(nn.Module):
    def __init__(self, num_inputs, hidden, num_actions, non_linearity):
        super(DuelingDQNHER, self).__init__()
        
        self.num_inputs = num_inputs * 2
        self.hidden = hidden
        self.num_actions = num_actions
        
        self.non_linearity = non_linearity
        
        self.sharedLayers = nn.ModuleList()
        self.sharedLayers.append(nn.Linear(self.num_inputs, self.hidden[0]))
        
        previous = self.hidden[0]
        for hidden_layer_size in self.hidden[1:-1]:
            self.sharedLayers.append(nn.Linear(previous, hidden_layer_size))
            previous = hidden_layer_size

        self.adv1 = nn.Linear(previous, self.hidden[-1])
        self.adv2 = nn.Linear(self.hidden[-1], self.num_actions)
        
        self.v1 = nn.Linear(previous, self.hidden[-1])
        self.v2 = nn.Linear(self.hidden[-1], 1)
        
    def forward(self, x):
        for i in range(len(self.sharedLayers)):
            
            x = self.non_linearity(self.sharedLayers[i](x))

        a = self.non_linearity(self.adv1(x))
        a = self.adv2(a)
        
        v = self.non_linearity(self.v1(x))
        v = self.v2(v)
        
        return v + a - a.mean()
    
    def increase_capacity(self, increment):
        for i in range(len(self.hidden)):
            self.hidden[i] += increment[i]
        
        # Check whether the increment isn't zero
        if increment[0] > 0:
            weight = self.sharedLayers[0].weight.data
            self.sharedLayers[0] = nn.Linear(self.num_inputs, self.hidden[0])
            self.sharedLayers[0].weight.data[0:-increment[0],:] = weight

        for i in range(1, len(self.sharedLayers)):
            weight = self.sharedLayers[i].weight.data
            self.sharedLayers[i] = nn.Linear(self.hidden[i-1], self.hidden[i])
            if increment[i] > 0:
                if increment[i-1] > 0:
                    self.sharedLayers[i].weight.data[0:-increment[i],0:-increment[i-1]] = weight
                else:
                    self.sharedLayers[i].weight.data[0:-increment[i],0:] = weight
            else:
                if increment[i-1] > 0:
                        self.sharedLayers[i].weight.data[0:,0:-increment[i-1]] = weight
                else:
                    self.sharedLayers[i].weight.data[0:,0:] = weight
            
        weight_adv1 = self.adv1.weight.data
        self.adv1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        
        weight_v1 = self.v1.weight.data
        self.v1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        if increment[-1] > 0:
            if increment[-2] > 0:
                self.adv1.weight.data[0:-increment[-1],0:-increment[-2]] = weight_adv1
                self.v1.weight.data[0:-increment[-1],0:-increment[-2]] = weight_v1
            else:
                self.adv1.weight.data[0:-increment[-1],0:] = weight_adv1
                self.v1.weight.data[0:-increment[-1],0:] = weight_v1
        else:
            if increment[-2] > 0:
                self.adv1.weight.data[0:,0:-increment[-2]] = weight_adv1
                self.v1.weight.data[0:,0:-increment[-2]] = weight_v1
            else:
                self.adv1.weight.data[0:,0:] = weight_adv1
                self.v1.weight.data[0:,0:] = weight_v1
            
        weight_adv2 = self.adv2.weight.data
        self.adv2 = nn.Linear(self.hidden[-1], self.num_actions)
        
        weight_v2 = self.v2.weight.data
        self.v2 = nn.Linear(self.hidden[-1], 1)
        
        if increment[-1] > 0:
            self.adv2.weight.data[:,0:-increment[-1]] = weight_adv2
            self.v2.weight.data[:,0:-increment[-1]] = weight_v2
        else:
            self.adv2.weight.data[:,0:] = weight_adv2
            self.v2.weight.data[:,0:] = weight_v2
        
    def decrease_capacity(self, increment):
        for i in range(len(self.hidden)):
            self.hidden[i] -= increment[i]

        weight = self.sharedLayers[0].weight.data
        self.sharedLayers[0] = nn.Linear(self.num_inputs, self.hidden[0])
        self.sharedLayers[0].weight.data = weight[0:-increment[0],:]
        
        for i in range(1, len(self.sharedLayers)):
            weight = self.sharedLayers[i].weight.data
            self.sharedLayers[i] = nn.Linear(self.hidden[i-1], self.hidden[i])
            self.sharedLayers[i].weight.data = weight[0:-increment[i],0:-increment[i-1]]
            
        weight = self.adv1.weight.data
        self.adv1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        self.adv1.weight.data = weight[0:-increment[-1],0:-increment[-2]]
            
        weight = self.adv2.weight.data
        self.adv2 = nn.Linear(self.hidden[-1], self.num_actions)
        self.adv2.weight.data = weight[:,0:-increment[-1]]
        
        weight = self.v1.weight.data
        self.v1 = nn.Linear(self.hidden[-2], self.hidden[-1])
        self.v1.weight.data = weight[0:-increment[-1],0:-increment[-2]]
            
        weight = self.v2.weight.data
        self.v2 = nn.Linear(self.hidden[-1], 1)
        self.v2.weight.data = weight[:,0:-increment[-1]]
        
    def act(self, state, goal, epsilon):
        if np.random.rand() > epsilon:
            state = torch.cat((torch.tensor([state], dtype=torch.float32, device=device),torch.tensor([goal], dtype=torch.float32, device=device)),1)
            q_values = self.forward(state)
            action = q_values.max(1)[1].view(1, 1).item()
        else:
            action =  np.random.randint(self.num_actions)
        return action

In [5]:
def evaluate_superflip(model, her):
    env = rubiks2.RubiksEnv2(2, unsolved_reward=-1.0)
    goal = env.get_observation()
    
    eval_model = torch.load(model)
    eval_model.to(device)
    
    max_tries = 1000
    solved = 0
    tries_solved = []
    cycles = 0
    
    for superflip in superflips:
        hashes = defaultdict(list)
        env.U = np.array(superflip[0])
        env.L = np.array(superflip[1])
        env.F = np.array(superflip[2])
        env.R = np.array(superflip[3])
        env.B = np.array(superflip[4])
        env.D = np.array(superflip[5])
        state = env.get_observation()
        done = 0
        tries = 0
        

        hashes = defaultdict(list)

        hashes[hash(state.tostring())] = [0,0,0,0,0,0]
        done = 0
        tries = 0
        
        while not done and tries < max_tries:
            mask = hashes[hash(state.tostring())]
            if her:
                action = eval_model.act(state, goal, 0)
            else:
                action = eval_model.act(state, 0, mask)
            
            next_state, reward, done, info = env.step(action)
            
            hstate = state.copy()
            state = next_state
            
            tries += 1
            
            h = hash(state.tostring())
            if h in hashes.keys():
                hashes[hash(hstate.tostring())][action] = -999
                cycles += 1
            else:
                hashes[h] = [0,0,0,0,0,0]
        if done:
            solved += 1
            tries_solved.append(tries)
    
    percentage_solved = solved / len(superflips)
    percentage_cycles = cycles
    average_tries = np.average(tries_solved)
    
    return percentage_solved, percentage_cycles, average_tries

def evaluate(model, scrambles, number_of_evaluations, her, seed=None):
    env = rubiks2.RubiksEnv2(2, unsolved_reward=-1.0, seed=seed)
    goal = env.get_observation()
    
    eval_model = torch.load(model)
    eval_model.to(device)
    
    max_tries = 1000
    
    solved = 0
    tries_solved = []
    cycles = 0
    
    for i in range(number_of_evaluations):
        hashes = defaultdict(list)
        state = env.reset(scrambles)
        hashes[hash(state.tostring())] = [0,0,0,0,0,0]
        done = 0
        tries = 0
        
        while not done and tries < max_tries:
            mask = hashes[hash(state.tostring())]
            if her:
                action = eval_model.act(state, goal, 0)
            else:
                action = eval_model.act(state, 0, mask)
            
            next_state, reward, done, info = env.step(action)
            
            hstate = state.copy()
            state = next_state
            
            tries += 1
            
            h = hash(state.tostring())
            if h in hashes.keys():
                hashes[hash(hstate.tostring())][action] = -999
                cycles += 1
            else:
                hashes[h] = [0,0,0,0,0,0]
        if done:
            solved += 1
            tries_solved.append(tries)
    
    percentage_solved = solved / number_of_evaluations
    percentage_cycles = cycles / number_of_evaluations
    average_tries = np.average(tries_solved)
    
    return percentage_solved, percentage_cycles, average_tries

models = []
models.append(('Baseline','./models/Baseline/model.pt', False))
models.append(('Mixed','./models/Mixed/model.pt', False))
models.append(('Joe','./models/Joe/model.pt', False))
models.append(('Sutskever','./models/Sutskever/model.pt', False))
models.append(('Prioritised0.5','./models/Prioritised0.5/model.pt', False))
models.append(('Prioritised0.7','./models/Prioritised0.7/model.pt', False))
models.append(('Prioritised0.9','./models/Prioritised0.9/model.pt', False))
models.append(('Dueling','./models/Dueling/model.pt', False))
models.append(('Double','./models/Double/model.pt', False))
models.append(('Linear All','./models/Linear_all/model.pt', False))
models.append(('Elu','./models/Elu/model.pt', False))
models.append(('Leaky Relu','./models/Leaky_relu/model.pt', False))
models.append(('Baseline HER','./models/Baseline_HER/model.pt', True))
models.append(('Mixed HER','./models/Mixed_HER/model.pt', True))
models.append(('Joe HER','./models/Joe_HER/model.pt', True))
models.append(('Sutskever HER','./models/Sutskever_HER/model.pt', True))
models.append(('Prioritised0.5 HER','./models/Prioritised0.5_HER/model.pt', True))
models.append(('Prioritised0.7 HER','./models/Prioritised0.7_HER/model.pt', True))
models.append(('Prioritised0.9 HER','./models/Prioritised0.9_HER/model.pt', True))
models.append(('Dueling HER','./models/Dueling_HER/model.pt', True))
models.append(('Double HER','./models/Double_HER/model.pt', True))
models.append(('Linear All HER','./models/Linear_all_HER/model.pt', True))
models.append(('Elu HER','./models/Elu_HER/model.pt', True))

for model in models:
    print(model[0])
#     percentage_solved, percentage_cycles, average_tries = evaluate_superflip(model[1], model[2])
    
#     print('Superflip: ', 
#           'Percentage Solved: ', percentage_solved,
#           'Percentage Cycles: ', percentage_cycles,
#           'Average Tries: ', average_tries)
    
    percentages_solved = []
    percentages_cycles = []
    averages_tries = []
    for scrambles in range(30):
        percentage_solved, percentage_cycles, average_tries = evaluate(model[1], scrambles + 1, 640, model[2], seed=42)
        print('Scrambles: ', scrambles + 1,
              'Percentage Solved: ', percentage_solved,
              'Percentage Cycles: ', percentage_cycles,
              'Average Tries: ', average_tries)
        percentages_solved.append(percentage_solved)
        percentages_cycles.append(percentage_cycles)
        averages_tries.append(average_tries)
    
    total_points = 0
    total_idx = 0
    for i, p in enumerate(percentages_solved):
        total_idx += i+1
        total_points += averages_tries[i]*p
    
    print('Normalised Score: ', total_points/total_idx)
        
    percentage_solved, percentage_cycles, average_tries = evaluate(model[1], 1000, 640, model[2], seed=42)
    print('Scrambles: ', 1000, 
          'Percentage Solved: ', percentage_solved,
          'Percentage Cycles: ', percentage_cycles,
          'Average Tries: ', average_tries)
        
    plt.figure()
    plt.plot([scrambles + 1 for scrambles in range(30)], percentages_solved)
    plt.plot([scrambles + 1 for scrambles in range(30)], percentages_cycles)
    plt.xlabel('Scramble Distance')
    plt.ylabel('Percentage')
    plt.legend(('Percentage Solved', 'Percentage Cycles'))
    plt.show()
    
    plt.figure()
    plt.plot([scrambles + 1 for scrambles in range(30)], averages_tries)
    plt.xlabel('Scramble Distance')
    plt.ylabel('Average Number of Tries')
    plt.show()

Baseline


NameError: name 'defaultdict' is not defined

In [ ]:
import os
for d in os.listdir('./models'):
    if d[0] is not '.':
        print(d)
        test('./models/' + d + '/model.pt')

In [ ]:
superflips = [[[[5,5],[0,4]],[[2,1],[5,4]],[[2,1],[3,5]],[[0,2],[3,2]],[[1,3],[0,1]],[[0,4],[4,3]]],
[[[4,5],[0,0]],[[5,1],[3,5]],[[2,4],[1,4]],[[1,1],[3,0]],[[4,3],[3,2]],[[2,0],[5,2]]],
[[[4,3],[0,3]],[[1,1],[0,3]],[[2,2],[4,0]],[[5,0],[4,5]],[[2,5],[1,4]],[[5,3],[1,2]]],
[[[3,3],[0,1]],[[5,1],[0,5]],[[2,5],[3,4]],[[2,4],[0,4]],[[0,2],[1,2]],[[4,1],[3,5]]],
[[[5,5],[0,0]],[[3,1],[3,1]],[[2,2],[4,4]],[[3,3],[1,1]],[[2,4],[2,4]],[[0,5],[0,5]]],
[[[0,1],[0,3]],[[3,1],[2,2]],[[2,4],[3,5]],[[0,0],[4,4]],[[4,2],[5,1]],[[5,1],[5,3]]],
[[[5,0],[0,3]],[[4,1],[0,2]],[[2,5],[3,2]],[[4,4],[5,3]],[[1,1],[2,3]],[[5,1],[4,0]]],
[[[4,0],[0,5]],[[5,1],[1,4]],[[2,3],[1,0]],[[2,2],[4,1]],[[3,3],[4,5]],[[5,3],[2,0]]],
[[[5,4],[0,4]],[[1,1],[2,2]],[[2,5],[0,3]],[[1,1],[4,4]],[[0,2],[3,5]],[[3,5],[3,0]]],
[[[5,2],[0,2]],[[1,1],[4,3]],[[2,5],[0,0]],[[3,3],[1,4]],[[0,2],[5,5]],[[4,4],[1,3]]],
[[[4,0],[0,5]],[[1,1],[4,0]],[[2,2],[3,5]],[[1,4],[2,3]],[[1,5],[0,3]],[[2,3],[5,4]]],
[[[4,5],[0,2]],[[0,1],[5,1]],[[2,5],[2,0]],[[3,4],[3,4]],[[3,1],[3,1]],[[5,2],[4,0]]],
[[[4,2],[0,0]],[[3,1],[5,4]],[[2,4],[1,1]],[[1,3],[2,5]],[[0,0],[3,3]],[[5,5],[2,4]]],
[[[5,2],[0,0]],[[1,1],[0,3]],[[2,4],[2,1]],[[1,5],[5,3]],[[3,2],[4,3]],[[0,4],[4,5]]],
[[[3,1],[0,0]],[[0,1],[3,2]],[[2,2],[5,5]],[[3,0],[2,1]],[[4,4],[5,5]],[[1,3],[4,4]]],
[[[5,2],[0,0]],[[3,1],[2,1]],[[2,3],[5,5]],[[4,3],[1,0]],[[0,4],[1,5]],[[4,2],[3,4]]],
[[[3,4],[0,5]],[[0,1],[0,3]],[[2,3],[2,1]],[[2,5],[2,5]],[[1,4],[3,4]],[[0,5],[1,4]]],
[[[1,3],[0,0]],[[4,1],[3,2]],[[2,2],[3,1]],[[3,4],[2,1]],[[0,0],[5,5]],[[5,5],[4,4]]],
[[[1,1],[0,5]],[[5,1],[4,3]],[[2,2],[5,0]],[[1,0],[3,4]],[[4,4],[5,0]],[[2,2],[3,3]]],
[[[3,4],[0,2]],[[0,1],[0,3]],[[2,5],[2,5]],[[3,5],[1,5]],[[1,4],[3,4]],[[0,2],[1,4]]],
[[[2,5],[0,4]],[[5,1],[3,2]],[[2,1],[3,4]],[[0,1],[5,0]],[[4,1],[4,0]],[[5,3],[2,3]]],
[[[2,4],[0,4]],[[5,1],[3,4]],[[2,5],[3,5]],[[1,3],[2,1]],[[5,1],[4,0]],[[0,3],[2,0]]],
[[[0,2],[0,4]],[[1,1],[3,5]],[[2,0],[4,4]],[[3,3],[5,1]],[[0,4],[2,2]],[[1,3],[5,5]]],
[[[2,2],[0,0]],[[5,1],[3,0]],[[2,2],[4,4]],[[3,5],[0,1]],[[3,1],[5,5]],[[3,1],[4,4]]],
[[[1,1],[0,5]],[[5,1],[5,3]],[[2,2],[2,2]],[[1,0],[3,0]],[[4,4],[4,4]],[[0,5],[3,3]]],
[[[0,2],[0,4]],[[1,1],[2,4]],[[2,0],[1,5]],[[3,3],[2,4]],[[0,4],[5,1]],[[5,3],[5,3]]],
[[[3,3],[0,5]],[[0,1],[0,3]],[[2,2],[2,2]],[[1,5],[3,5]],[[4,4],[4,4]],[[0,5],[1,1]]],
[[[0,5],[0,4]],[[3,1],[1,5]],[[2,5],[2,3]],[[1,2],[0,3]],[[1,2],[4,0]],[[3,4],[4,5]]],
[[[0,5],[0,5]],[[1,1],[3,3]],[[2,2],[4,4]],[[1,3],[1,3]],[[2,4],[2,4]],[[5,5],[0,0]]],
[[[2,5],[0,4]],[[5,1],[3,0]],[[2,0],[3,4]],[[3,4],[0,4]],[[3,1],[1,2]],[[2,1],[5,5]]],
[[[1,5],[0,4]],[[5,1],[3,0]],[[2,1],[3,4]],[[0,2],[3,3]],[[1,4],[5,5]],[[2,0],[4,2]]],
[[[0,2],[0,4]],[[1,1],[2,4]],[[2,0],[1,3]],[[3,3],[5,5]],[[0,4],[3,1]],[[5,2],[5,4]]],
[[[0,1],[0,5]],[[4,1],[5,5]],[[2,4],[2,1]],[[3,0],[5,2]],[[4,3],[0,2]],[[3,4],[1,3]]],
[[[0,1],[0,5]],[[3,1],[2,1]],[[2,2],[4,0]],[[1,4],[4,3]],[[5,2],[4,5]],[[0,3],[3,5]]],
[[[1,3],[0,2]],[[5,1],[0,1]],[[2,5],[2,0]],[[3,4],[3,4]],[[0,4],[5,4]],[[5,2],[1,3]]],
[[[3,1],[0,0]],[[0,1],[3,3]],[[2,2],[4,5]],[[3,0],[1,1]],[[4,4],[5,2]],[[5,2],[5,4]]],
[[[0,2],[0,2]],[[3,1],[4,4]],[[2,1],[3,5]],[[5,5],[3,1]],[[3,2],[5,1]],[[0,4],[0,4]]],
[[[3,5],[0,5]],[[2,1],[5,4]],[[2,1],[3,5]],[[4,2],[2,0]],[[1,0],[1,4]],[[0,3],[3,4]]],
[[[1,0],[0,3]],[[2,1],[4,4]],[[2,0],[5,2]],[[2,3],[3,4]],[[4,5],[0,5]],[[3,5],[1,1]]],
[[[0,5],[0,5]],[[3,1],[0,0]],[[2,3],[1,4]],[[2,2],[5,5]],[[1,2],[4,3]],[[4,3],[4,1]]],
[[[2,5],[0,4]],[[5,1],[3,0]],[[2,1],[4,2]],[[0,1],[0,2]],[[4,1],[3,5]],[[3,3],[4,5]]],
[[[4,2],[0,4]],[[5,1],[3,4]],[[2,5],[0,5]],[[1,1],[2,0]],[[5,3],[4,0]],[[1,3],[2,3]]],
[[[1,1],[0,0]],[[5,1],[2,3]],[[2,4],[0,0]],[[1,5],[3,4]],[[2,4],[5,5]],[[4,2],[3,3]]],
[[[3,0],[0,4]],[[0,1],[2,5]],[[2,3],[4,2]],[[5,4],[5,3]],[[1,4],[2,5]],[[1,1],[3,0]]],
[[[1,1],[0,0]],[[2,1],[5,4]],[[2,4],[3,3]],[[1,4],[2,5]],[[5,5],[2,4]],[[0,0],[3,3]]],
[[[5,3],[0,0]],[[4,1],[3,0]],[[2,4],[4,5]],[[1,5],[1,2]],[[4,1],[0,2]],[[3,2],[5,3]]],
[[[2,3],[0,0]],[[0,1],[3,5]],[[2,4],[4,5]],[[1,4],[1,5]],[[0,3],[2,5]],[[1,2],[4,3]]],
[[[5,5],[0,0]],[[1,1],[3,3]],[[2,2],[4,4]],[[3,1],[3,1]],[[4,2],[4,2]],[[5,0],[5,0]]],
[[[1,2],[0,3]],[[5,1],[1,4]],[[2,4],[5,2]],[[0,5],[0,1]],[[3,4],[2,0]],[[3,3],[4,5]]],
[[[3,5],[0,1]],[[0,1],[0,3]],[[2,5],[2,5]],[[2,4],[2,4]],[[3,4],[1,4]],[[0,3],[1,5]]],
[[[3,4],[0,4]],[[5,1],[4,0]],[[2,0],[3,2]],[[3,5],[5,0]],[[1,2],[1,3]],[[2,1],[5,4]]],
[[[0,4],[0,2]],[[1,1],[2,4]],[[2,3],[1,4]],[[0,0],[5,5]],[[3,4],[2,1]],[[5,3],[5,3]]],
[[[1,2],[0,2]],[[2,1],[0,4]],[[2,3],[1,4]],[[0,5],[5,4]],[[3,5],[3,4]],[[5,3],[1,0]]],
[[[3,1],[0,0]],[[5,1],[3,4]],[[2,2],[0,0]],[[3,5],[4,1]],[[2,2],[5,5]],[[1,3],[4,4]]],
[[[0,5],[0,4]],[[3,1],[2,2]],[[2,1],[3,1]],[[0,4],[5,4]],[[3,2],[3,1]],[[5,4],[5,0]]],
[[[3,1],[0,1]],[[0,1],[0,3]],[[2,4],[2,5]],[[5,5],[2,4]],[[2,4],[5,4]],[[0,3],[1,3]]],
[[[3,3],[0,1]],[[5,1],[5,4]],[[2,5],[3,4]],[[2,5],[0,3]],[[4,2],[2,1]],[[0,1],[4,0]]],
[[[3,3],[0,0]],[[2,1],[3,4]],[[2,4],[1,1]],[[1,4],[2,3]],[[0,0],[5,5]],[[5,5],[4,2]]],
[[[0,3],[0,3]],[[1,1],[2,4]],[[2,0],[1,4]],[[2,4],[5,5]],[[0,4],[2,1]],[[5,3],[5,3]]],
[[[2,4],[0,4]],[[5,1],[3,4]],[[2,5],[5,0]],[[1,0],[1,2]],[[3,1],[3,0]],[[3,4],[2,5]]],
[[[2,0],[0,5]],[[5,1],[3,0]],[[2,1],[3,4]],[[4,4],[5,3]],[[1,1],[0,2]],[[2,3],[5,4]]],
[[[0,4],[0,3]],[[3,1],[1,4]],[[2,2],[3,4]],[[5,5],[0,3]],[[1,2],[4,5]],[[0,1],[2,5]]],
[[[0,2],[0,4]],[[1,1],[1,1]],[[2,0],[2,5]],[[3,3],[3,3]],[[0,4],[5,4]],[[5,4],[5,2]]],
[[[3,3],[0,0]],[[2,1],[5,4]],[[2,4],[1,1]],[[1,4],[2,5]],[[0,0],[2,4]],[[5,5],[3,3]]],
[[[5,3],[0,4]],[[4,1],[2,1]],[[2,0],[4,0]],[[3,2],[3,4]],[[5,1],[5,1]],[[0,2],[5,3]]],
[[[0,1],[0,1]],[[1,1],[3,3]],[[2,5],[0,4]],[[2,4],[5,5]],[[5,4],[2,0]],[[4,3],[2,3]]],
[[[0,2],[0,4]],[[1,1],[4,2]],[[2,0],[5,4]],[[3,3],[5,5]],[[0,4],[2,5]],[[1,3],[1,3]]],
[[[3,5],[0,1]],[[2,1],[5,1]],[[2,5],[5,0]],[[2,3],[1,4]],[[2,0],[3,4]],[[4,4],[3,0]]],
[[[4,5],[0,0]],[[3,1],[3,2]],[[2,2],[3,1]],[[3,1],[2,1]],[[4,0],[4,5]],[[5,5],[4,0]]],
[[[2,2],[0,5]],[[3,1],[0,4]],[[2,2],[1,1]],[[1,3],[4,5]],[[0,5],[3,3]],[[5,0],[4,4]]],
[[[0,5],[0,3]],[[3,1],[3,2]],[[2,5],[5,4]],[[4,3],[1,4]],[[2,2],[0,4]],[[1,5],[0,1]]],
[[[1,3],[0,4]],[[5,1],[5,1]],[[2,0],[4,0]],[[3,2],[3,4]],[[5,4],[5,2]],[[0,2],[1,3]]],
[[[1,5],[0,3]],[[2,1],[5,1]],[[2,4],[4,3]],[[0,4],[2,5]],[[3,5],[2,1]],[[0,0],[4,3]]],
[[[5,1],[0,1]],[[1,1],[2,2]],[[2,4],[3,0]],[[5,0],[4,4]],[[4,2],[5,3]],[[5,3],[0,3]]],
[[[2,2],[0,5]],[[0,1],[3,5]],[[2,2],[4,4]],[[1,5],[0,3]],[[3,3],[0,5]],[[1,1],[4,4]]],
[[[3,1],[0,0]],[[0,1],[3,3]],[[2,2],[5,4]],[[3,0],[1,1]],[[4,4],[2,5]],[[2,5],[4,5]]],
[[[4,0],[0,2]],[[0,1],[1,5]],[[2,1],[3,3]],[[5,3],[5,0]],[[4,1],[3,4]],[[4,2],[5,2]]],
[[[3,5],[0,1]],[[2,1],[0,4]],[[2,5],[3,5]],[[2,3],[3,1]],[[2,0],[5,4]],[[0,4],[1,4]]],
[[[5,1],[0,4]],[[2,1],[2,3]],[[2,0],[2,0]],[[3,4],[1,4]],[[5,3],[5,1]],[[0,4],[5,3]]],
[[[3,5],[0,1]],[[2,1],[3,4]],[[2,5],[1,4]],[[2,3],[0,3]],[[2,0],[0,5]],[[5,1],[4,4]]],
[[[3,1],[0,3]],[[4,1],[0,4]],[[2,2],[1,0]],[[5,5],[4,0]],[[2,5],[3,4]],[[5,3],[1,2]]],
[[[3,3],[0,1]],[[4,1],[1,2]],[[2,0],[5,2]],[[4,4],[3,2]],[[0,5],[0,4]],[[1,5],[5,3]]],
[[[3,5],[0,5]],[[2,1],[5,4]],[[2,4],[3,4]],[[3,2],[0,2]],[[1,0],[3,1]],[[0,1],[4,5]]],
[[[1,1],[0,5]],[[5,1],[4,0]],[[2,2],[3,3]],[[1,0],[5,4]],[[4,4],[3,3]],[[2,2],[5,0]]],
[[[0,4],[0,4]],[[3,1],[2,2]],[[2,0],[3,5]],[[3,1],[4,4]],[[0,2],[5,1]],[[5,1],[5,3]]],
[[[3,5],[0,2]],[[2,1],[3,4]],[[2,5],[0,4]],[[3,2],[5,5]],[[1,0],[4,4]],[[1,3],[0,1]]],
[[[1,4],[0,2]],[[5,1],[0,1]],[[2,5],[2,0]],[[3,0],[3,5]],[[3,4],[3,4]],[[5,2],[1,4]]],
[[[5,4],[0,5]],[[4,1],[3,2]],[[2,2],[0,3]],[[1,1],[5,4]],[[0,1],[3,5]],[[3,2],[4,0]]],
[[[1,1],[0,5]],[[5,1],[4,2]],[[2,2],[3,3]],[[1,0],[2,4]],[[4,4],[5,0]],[[5,0],[3,3]]],
[[[0,5],[0,5]],[[1,1],[3,3]],[[2,4],[2,4]],[[3,1],[3,1]],[[4,4],[2,2]],[[0,0],[5,5]]],
[[[2,3],[0,0]],[[5,1],[4,1]],[[2,3],[4,2]],[[4,5],[0,2]],[[4,1],[3,5]],[[0,3],[1,5]]],
[[[4,5],[0,2]],[[0,1],[0,2]],[[2,5],[0,1]],[[3,4],[2,1]],[[3,1],[5,3]],[[3,5],[4,4]]],
[[[5,2],[0,0]],[[3,1],[2,1]],[[2,3],[5,3]],[[4,5],[2,4]],[[3,4],[0,1]],[[4,0],[5,1]]],
[[[1,2],[0,1]],[[5,1],[3,4]],[[2,5],[3,4]],[[2,3],[0,3]],[[0,4],[4,2]],[[0,1],[5,5]]],
[[[5,5],[0,0]],[[2,1],[3,4]],[[2,2],[0,0]],[[3,2],[4,1]],[[1,3],[5,5]],[[1,3],[4,4]]],
[[[1,1],[0,5]],[[5,1],[0,0]],[[2,2],[3,4]],[[1,0],[5,5]],[[4,4],[2,3]],[[2,3],[4,3]]],
[[[0,3],[0,4]],[[4,1],[1,3]],[[2,5],[2,4]],[[1,2],[0,3]],[[5,3],[4,5]],[[0,1],[2,5]]],
[[[2,3],[0,4]],[[0,1],[0,1]],[[2,5],[4,5]],[[1,5],[1,5]],[[4,3],[2,3]],[[0,2],[4,3]]],
[[[0,0],[0,0]],[[1,1],[2,4]],[[2,2],[1,3]],[[3,3],[4,2]],[[4,4],[3,1]],[[5,5],[5,5]]],
[[[0,5],[0,4]],[[4,1],[5,1]],[[2,5],[4,2]],[[1,2],[0,3]],[[1,3],[4,3]],[[0,3],[2,5]]],
[[[1,5],[0,3]],[[2,1],[2,3]],[[2,4],[2,4]],[[0,4],[0,4]],[[3,5],[1,5]],[[0,1],[3,5]]],
[[[3,1],[0,0]],[[0,1],[3,5]],[[2,2],[1,3]],[[3,0],[5,1]],[[4,4],[5,5]],[[2,2],[4,4]]],
[[[5,5],[0,4]],[[3,1],[3,4]],[[2,1],[3,5]],[[0,2],[2,4]],[[1,4],[1,0]],[[0,3],[2,5]]],
[[[5,5],[0,0]],[[4,1],[3,2]],[[2,4],[0,0]],[[1,2],[4,3]],[[1,1],[5,5]],[[3,3],[4,2]]],
[[[4,4],[0,0]],[[0,1],[3,2]],[[2,2],[3,1]],[[3,0],[2,1]],[[3,1],[5,5]],[[5,5],[4,4]]],
[[[0,1],[0,2]],[[4,1],[2,4]],[[2,5],[5,1]],[[3,4],[2,0]],[[5,3],[1,3]],[[3,5],[0,4]]],
[[[3,4],[0,1]],[[5,1],[3,3]],[[2,5],[4,4]],[[2,0],[0,5]],[[3,2],[4,0]],[[5,1],[2,1]]],
[[[5,2],[0,2]],[[1,1],[1,1]],[[2,5],[4,5]],[[3,3],[3,3]],[[0,2],[0,4]],[[0,4],[5,4]]],
[[[3,5],[0,4]],[[5,1],[4,4]],[[2,1],[3,3]],[[0,2],[4,3]],[[1,2],[2,5]],[[0,5],[1,0]]],
[[[3,3],[0,0]],[[0,1],[5,4]],[[2,4],[1,1]],[[1,0],[2,5]],[[2,4],[3,3]],[[5,5],[2,4]]],
[[[2,5],[0,4]],[[5,1],[5,1]],[[2,0],[4,0]],[[3,4],[3,2]],[[3,1],[3,1]],[[0,2],[4,5]]],
[[[3,3],[0,4]],[[2,1],[2,5]],[[2,5],[1,4]],[[1,5],[0,3]],[[4,0],[0,5]],[[2,1],[3,4]]],
[[[1,1],[0,5]],[[5,1],[0,0]],[[2,2],[4,3]],[[1,0],[5,5]],[[4,4],[3,2]],[[3,2],[3,4]]],
[[[1,3],[0,0]],[[5,1],[0,1]],[[2,2],[2,2]],[[3,5],[3,0]],[[4,4],[4,4]],[[5,5],[1,3]]],
[[[0,5],[0,5]],[[1,1],[0,0]],[[2,1],[3,4]],[[4,2],[5,5]],[[1,4],[2,3]],[[2,3],[4,3]]],
[[[3,5],[0,4]],[[5,1],[0,1]],[[2,1],[2,1]],[[0,4],[5,4]],[[3,2],[3,2]],[[5,4],[3,0]]],
[[[0,3],[0,5]],[[1,1],[2,4]],[[2,4],[1,4]],[[3,0],[3,5]],[[2,4],[2,1]],[[5,0],[5,3]]],
[[[2,5],[0,4]],[[5,1],[5,4]],[[2,1],[3,0]],[[0,1],[3,5]],[[4,1],[2,4]],[[0,2],[3,3]]],
[[[0,4],[0,4]],[[3,1],[2,2]],[[2,1],[3,5]],[[0,0],[3,1]],[[3,2],[5,1]],[[5,4],[5,4]]],
[[[3,4],[0,0]],[[5,1],[3,1]],[[2,3],[4,2]],[[4,5],[5,3]],[[1,2],[4,0]],[[0,1],[2,5]]],
[[[3,5],[0,1]],[[2,1],[2,1]],[[2,4],[2,4]],[[5,4],[0,4]],[[3,0],[3,5]],[[5,1],[3,0]]],
[[[2,2],[0,0]],[[5,1],[3,4]],[[2,2],[3,1]],[[3,5],[4,1]],[[3,1],[5,5]],[[0,0],[4,4]]],
[[[4,1],[0,4]],[[5,1],[0,5]],[[2,0],[2,1]],[[3,0],[5,2]],[[4,3],[5,2]],[[3,4],[3,1]]],
[[[0,4],[0,1]],[[3,1],[5,3]],[[2,0],[5,4]],[[4,0],[5,1]],[[3,2],[5,2]],[[2,3],[1,4]]],
[[[2,3],[0,4]],[[3,1],[3,4]],[[2,5],[0,3]],[[1,4],[4,5]],[[0,5],[1,0]],[[1,5],[2,2]]],
[[[3,5],[0,4]],[[2,1],[4,4]],[[2,3],[3,1]],[[5,2],[4,3]],[[1,0],[5,5]],[[0,0],[1,2]]],
[[[2,1],[0,4]],[[0,1],[2,2]],[[2,0],[5,4]],[[3,0],[5,1]],[[4,3],[5,5]],[[1,3],[3,4]]],
[[[3,0],[0,4]],[[4,1],[1,3]],[[2,0],[5,1]],[[3,2],[5,1]],[[3,5],[4,5]],[[2,4],[2,0]]],
[[[3,5],[0,4]],[[2,1],[2,1]],[[2,5],[2,0]],[[1,4],[1,4]],[[3,0],[3,5]],[[5,4],[3,0]]],
[[[5,2],[0,2]],[[1,1],[4,4]],[[2,5],[0,3]],[[3,3],[4,4]],[[0,2],[3,5]],[[1,5],[1,0]]],
[[[0,0],[0,0]],[[3,1],[4,4]],[[2,4],[1,1]],[[1,3],[2,2]],[[4,2],[3,3]],[[5,5],[5,5]]],
[[[3,3],[0,4]],[[2,1],[3,4]],[[2,5],[0,1]],[[1,5],[2,5]],[[4,0],[2,4]],[[1,5],[0,3]]],
[[[3,5],[0,4]],[[4,1],[2,4]],[[2,1],[3,3]],[[0,2],[5,5]],[[1,5],[4,3]],[[0,2],[0,1]]],
[[[1,5],[0,4]],[[4,1],[0,0]],[[2,5],[3,3]],[[1,2],[4,3]],[[1,0],[5,3]],[[2,5],[4,2]]],
[[[3,4],[0,1]],[[5,1],[4,4]],[[2,5],[1,2]],[[2,3],[0,4]],[[5,2],[0,0]],[[5,3],[3,1]]],
[[[2,1],[0,4]],[[3,1],[1,4]],[[2,0],[5,1]],[[3,0],[5,0]],[[4,5],[3,5]],[[3,4],[2,2]]],
[[[3,4],[0,4]],[[5,1],[2,0]],[[2,1],[3,4]],[[0,3],[3,1]],[[5,2],[5,1]],[[2,0],[5,4]]],
[[[3,5],[0,5]],[[5,1],[4,0]],[[2,4],[3,4]],[[3,2],[0,0]],[[1,2],[4,5]],[[2,1],[1,3]]],
[[[1,5],[0,4]],[[5,1],[3,4]],[[2,1],[3,1]],[[0,3],[2,3]],[[2,4],[4,0]],[[0,5],[2,5]]],
[[[5,0],[0,3]],[[1,1],[5,4]],[[2,2],[0,3]],[[5,2],[4,0]],[[3,2],[4,1]],[[1,5],[4,3]]],
[[[5,0],[0,4]],[[1,1],[4,5]],[[2,5],[2,4]],[[1,2],[0,3]],[[3,2],[4,0]],[[3,1],[3,5]]],
[[[3,1],[0,3]],[[5,1],[5,0]],[[2,5],[3,4]],[[4,5],[0,3]],[[2,2],[0,1]],[[2,1],[4,4]]],
[[[5,0],[0,3]],[[1,1],[3,4]],[[2,2],[3,5]],[[5,4],[4,3]],[[1,2],[4,0]],[[0,1],[2,5]]],
[[[0,4],[0,5]],[[3,1],[2,2]],[[2,1],[3,1]],[[4,0],[4,5]],[[3,2],[3,1]],[[5,0],[5,4]]],
[[[5,3],[0,5]],[[1,1],[4,0]],[[2,3],[1,4]],[[2,0],[5,4]],[[2,2],[3,5]],[[4,3],[1,0]]],
[[[3,2],[0,1]],[[0,1],[3,5]],[[2,0],[1,1]],[[4,3],[5,5]],[[0,4],[2,5]],[[2,4],[4,3]]],
[[[1,5],[0,2]],[[4,1],[4,1]],[[2,5],[2,0]],[[3,4],[3,4]],[[3,0],[3,5]],[[5,2],[1,0]]],
[[[1,1],[0,2]],[[4,1],[3,2]],[[2,1],[3,0]],[[5,4],[3,4]],[[5,0],[3,5]],[[5,2],[4,0]]],
[[[0,3],[0,4]],[[3,1],[1,5]],[[2,1],[2,5]],[[0,4],[3,5]],[[0,2],[4,5]],[[3,4],[2,1]]],
[[[4,1],[0,2]],[[3,1],[1,5]],[[2,5],[3,2]],[[3,4],[0,1]],[[5,0],[2,0]],[[4,3],[4,5]]],
[[[5,5],[0,0]],[[1,1],[3,3]],[[2,4],[2,4]],[[1,3],[1,3]],[[2,2],[4,4]],[[0,5],[0,5]]],
[[[3,5],[0,0]],[[5,1],[1,3]],[[2,3],[2,5]],[[4,2],[4,3]],[[1,2],[4,0]],[[0,1],[4,5]]],
[[[3,1],[0,0]],[[0,1],[4,2]],[[2,2],[3,1]],[[3,0],[2,4]],[[4,4],[5,5]],[[5,5],[1,3]]],
[[[1,0],[0,3]],[[4,1],[1,2]],[[2,5],[0,5]],[[4,3],[2,2]],[[4,0],[5,4]],[[3,3],[5,1]]],
[[[0,4],[0,1]],[[4,1],[2,4]],[[2,5],[0,4]],[[2,3],[1,5]],[[5,3],[2,3]],[[1,5],[0,3]]],
[[[0,2],[0,4]],[[1,1],[0,0]],[[2,5],[3,4]],[[1,1],[5,5]],[[5,4],[2,3]],[[2,3],[4,3]]],
[[[0,2],[0,4]],[[1,1],[5,5]],[[2,0],[1,4]],[[3,3],[5,5]],[[0,4],[2,1]],[[2,3],[4,3]]],
[[[0,1],[0,1]],[[1,1],[2,4]],[[2,5],[3,4]],[[2,4],[5,5]],[[5,4],[2,3]],[[0,3],[0,3]]],
[[[3,5],[0,0]],[[2,1],[2,4]],[[2,4],[1,1]],[[1,4],[2,4]],[[3,0],[3,5]],[[5,5],[3,0]]],
[[[3,2],[0,0]],[[4,1],[3,5]],[[2,4],[4,3]],[[1,1],[2,3]],[[5,5],[0,2]],[[1,0],[5,4]]],
[[[2,3],[0,4]],[[5,1],[3,3]],[[2,5],[4,4]],[[1,4],[0,3]],[[0,1],[5,0]],[[5,1],[2,2]]],
[[[0,0],[0,0]],[[3,1],[2,2]],[[2,4],[3,3]],[[1,3],[4,4]],[[4,2],[1,1]],[[5,5],[5,5]]],
[[[3,1],[0,0]],[[0,1],[5,2]],[[2,2],[3,1]],[[3,0],[2,5]],[[4,4],[3,1]],[[5,5],[4,4]]],
[[[4,5],[0,3]],[[5,1],[2,1]],[[2,4],[4,5]],[[0,3],[1,4]],[[2,3],[1,3]],[[0,2],[0,5]]],
[[[0,4],[0,2]],[[1,1],[1,1]],[[2,5],[2,0]],[[3,3],[3,3]],[[5,4],[0,4]],[[5,2],[5,4]]],
[[[0,5],[0,5]],[[3,1],[3,1]],[[2,4],[2,4]],[[3,3],[1,1]],[[2,2],[4,4]],[[5,5],[0,0]]],
[[[0,2],[0,2]],[[3,1],[0,0]],[[2,5],[4,1]],[[3,1],[5,5]],[[5,2],[3,4]],[[3,4],[1,4]]],
[[[0,4],[0,4]],[[3,1],[2,2]],[[2,1],[3,4]],[[0,0],[5,5]],[[3,2],[4,1]],[[5,3],[5,1]]],
[[[1,5],[0,5]],[[5,1],[0,1]],[[2,4],[2,3]],[[3,3],[2,4]],[[2,4],[3,4]],[[5,0],[1,0]]],
[[[4,3],[0,3]],[[1,1],[3,4]],[[2,5],[0,1]],[[4,2],[2,0]],[[5,5],[4,0]],[[1,5],[2,3]]],
[[[3,1],[0,1]],[[5,1],[3,4]],[[2,5],[1,4]],[[2,0],[3,3]],[[4,2],[4,0]],[[5,0],[2,5]]],
[[[3,5],[0,1]],[[2,1],[3,4]],[[2,4],[0,2]],[[5,4],[5,3]],[[3,0],[5,4]],[[1,1],[0,2]]],
[[[4,1],[0,2]],[[5,1],[1,3]],[[2,5],[0,1]],[[3,4],[2,2]],[[5,3],[0,0]],[[4,5],[4,3]]],
[[[3,5],[0,1]],[[2,1],[1,3]],[[2,5],[0,4]],[[2,3],[5,4]],[[2,0],[1,0]],[[4,3],[4,5]]],
[[[2,5],[0,3]],[[3,1],[4,3]],[[2,4],[2,5]],[[0,1],[3,2]],[[4,5],[5,1]],[[0,4],[0,1]]],
[[[3,2],[0,1]],[[4,1],[4,5]],[[2,5],[2,4]],[[2,3],[0,4]],[[0,5],[1,0]],[[3,1],[3,5]]],
[[[4,5],[0,4]],[[0,1],[3,3]],[[2,3],[2,4]],[[5,1],[3,5]],[[4,1],[1,2]],[[0,0],[5,2]]],
[[[3,4],[0,4]],[[5,1],[3,4]],[[2,5],[0,5]],[[1,0],[3,1]],[[3,2],[2,0]],[[1,4],[2,5]]],
[[[1,5],[0,3]],[[5,1],[2,0]],[[2,0],[1,0]],[[2,4],[4,5]],[[3,4],[2,1]],[[4,3],[5,3]]],
[[[0,2],[0,4]],[[1,1],[5,3]],[[2,0],[4,4]],[[3,3],[1,5]],[[0,4],[2,2]],[[5,5],[1,3]]],
[[[3,5],[0,4]],[[2,1],[0,2]],[[2,5],[3,4]],[[1,2],[3,3]],[[1,0],[4,4]],[[5,0],[1,5]]],
[[[2,3],[0,4]],[[5,1],[3,3]],[[2,5],[0,3]],[[1,5],[5,4]],[[4,1],[0,0]],[[4,2],[2,1]]],
[[[5,2],[0,2]],[[1,1],[4,4]],[[2,5],[1,3]],[[3,3],[0,5]],[[0,2],[3,1]],[[5,4],[0,4]]],
[[[2,1],[0,4]],[[5,1],[3,0]],[[2,5],[3,3]],[[1,0],[4,4]],[[4,1],[3,2]],[[2,5],[5,0]]],
[[[3,0],[0,5]],[[5,1],[3,1]],[[2,2],[5,3]],[[1,4],[0,3]],[[1,2],[4,0]],[[4,4],[2,5]]],
[[[4,5],[0,2]],[[3,1],[4,3]],[[2,1],[2,5]],[[5,4],[2,1]],[[3,0],[5,1]],[[0,3],[0,4]]],
[[[3,3],[0,4]],[[5,1],[5,0]],[[2,5],[3,4]],[[1,4],[0,1]],[[0,2],[2,4]],[[2,1],[3,5]]],
[[[1,2],[0,0]],[[5,1],[2,5]],[[2,3],[3,1]],[[4,5],[4,2]],[[3,4],[0,1]],[[4,0],[5,3]]],
[[[3,5],[0,2]],[[5,1],[4,4]],[[2,3],[0,0]],[[0,2],[4,5]],[[1,2],[4,3]],[[1,3],[5,1]]],
[[[0,0],[0,0]],[[1,1],[4,2]],[[2,2],[3,1]],[[3,3],[2,4]],[[4,4],[1,3]],[[5,5],[5,5]]],
[[[2,2],[0,4]],[[5,1],[3,2]],[[2,5],[3,4]],[[1,3],[0,3]],[[0,1],[4,4]],[[5,1],[0,5]]],
[[[3,5],[0,4]],[[2,1],[4,4]],[[2,5],[3,3]],[[1,2],[5,1]],[[1,0],[4,3]],[[0,2],[5,0]]],
[[[5,5],[0,2]],[[3,1],[4,1]],[[2,5],[2,0]],[[3,1],[3,4]],[[4,4],[3,1]],[[5,2],[0,0]]],
[[[0,5],[0,4]],[[4,1],[5,5]],[[2,1],[3,0]],[[0,2],[3,5]],[[1,3],[4,3]],[[4,2],[2,1]]],
[[[4,5],[0,1]],[[3,1],[5,1]],[[2,4],[4,3]],[[5,3],[2,5]],[[2,0],[1,4]],[[0,0],[3,2]]],
[[[3,4],[0,1]],[[0,1],[0,3]],[[2,5],[2,5]],[[2,3],[2,1]],[[5,4],[5,4]],[[0,3],[1,4]]],
[[[3,4],[0,1]],[[5,1],[4,0]],[[2,5],[4,4]],[[2,3],[0,0]],[[5,2],[3,5]],[[3,1],[1,2]]],
[[[3,5],[0,1]],[[0,1],[3,4]],[[2,5],[0,4]],[[2,4],[1,2]],[[3,4],[3,0]],[[1,5],[2,5]]],
[[[3,5],[0,4]],[[5,1],[3,4]],[[2,0],[0,1]],[[3,1],[2,3]],[[4,2],[4,0]],[[1,5],[2,5]]],
[[[3,4],[0,4]],[[5,1],[3,2]],[[2,1],[5,1]],[[0,3],[5,0]],[[5,2],[4,0]],[[1,4],[2,3]]],
[[[3,5],[0,4]],[[5,1],[0,0]],[[2,0],[3,4]],[[3,2],[1,3]],[[1,2],[4,4]],[[2,5],[1,5]]],
[[[5,5],[0,2]],[[1,1],[3,5]],[[2,3],[3,4]],[[0,1],[3,0]],[[4,2],[1,2]],[[4,0],[5,4]]],
[[[0,1],[0,5]],[[4,1],[4,5]],[[2,4],[2,1]],[[3,4],[2,0]],[[5,3],[3,1]],[[3,5],[0,2]]],
[[[1,0],[0,4]],[[5,1],[5,4]],[[2,1],[3,5]],[[0,2],[3,5]],[[3,4],[1,3]],[[0,4],[2,2]]],
[[[1,5],[0,5]],[[5,1],[0,1]],[[2,3],[2,4]],[[2,4],[3,3]],[[3,4],[2,4]],[[5,0],[1,0]]],
[[[5,3],[0,0]],[[2,1],[2,4]],[[2,4],[1,1]],[[1,4],[2,4]],[[0,3],[5,3]],[[5,5],[0,3]]],
[[[3,1],[0,1]],[[0,1],[0,3]],[[2,5],[2,4]],[[2,4],[5,5]],[[5,4],[2,4]],[[0,3],[1,3]]],
[[[4,3],[0,0]],[[1,1],[3,4]],[[2,4],[3,2]],[[1,2],[5,3]],[[5,5],[4,0]],[[0,1],[2,5]]],
[[[4,4],[0,3]],[[5,1],[2,0]],[[2,2],[3,4]],[[5,5],[0,3]],[[1,3],[0,1]],[[2,1],[5,4]]],
[[[1,2],[0,3]],[[5,1],[2,0]],[[2,5],[4,2]],[[4,5],[0,0]],[[3,4],[1,1]],[[3,3],[5,4]]],
[[[3,5],[0,4]],[[0,1],[3,4]],[[2,5],[0,0]],[[1,2],[3,3]],[[1,4],[4,2]],[[1,2],[5,5]]],
[[[4,2],[0,4]],[[1,1],[0,2]],[[2,3],[3,0]],[[5,1],[3,4]],[[5,5],[0,3]],[[5,2],[4,1]]],
[[[2,5],[0,2]],[[0,1],[3,0]],[[2,1],[1,4]],[[5,1],[3,3]],[[4,3],[4,2]],[[4,0],[5,5]]],
[[[1,5],[0,1]],[[2,1],[3,2]],[[2,0],[0,4]],[[4,3],[1,3]],[[2,5],[4,4]],[[3,5],[0,5]]],
[[[1,5],[0,5]],[[4,1],[3,4]],[[2,2],[5,4]],[[1,3],[1,2]],[[2,0],[0,4]],[[3,5],[0,3]]],
[[[3,2],[0,5]],[[0,1],[0,1]],[[2,1],[4,1]],[[4,5],[2,5]],[[3,4],[3,2]],[[0,5],[3,4]]],
[[[5,4],[0,0]],[[3,1],[3,2]],[[2,2],[3,1]],[[3,1],[2,1]],[[0,4],[5,4]],[[5,5],[0,4]]],
[[[5,1],[0,1]],[[1,1],[3,3]],[[2,0],[5,4]],[[4,4],[5,0]],[[5,2],[4,0]],[[2,3],[2,3]]],
[[[4,2],[0,4]],[[1,1],[4,0]],[[2,1],[3,4]],[[0,1],[3,5]],[[5,5],[2,3]],[[2,0],[5,3]]],
[[[0,4],[0,5]],[[3,1],[5,3]],[[2,3],[0,4]],[[2,3],[1,0]],[[5,2],[1,2]],[[4,5],[1,4]]],
[[[3,4],[0,1]],[[5,1],[5,3]],[[2,0],[2,0]],[[4,3],[4,1]],[[5,2],[5,2]],[[0,3],[1,4]]],
[[[5,4],[0,5]],[[1,1],[0,4]],[[2,3],[5,4]],[[2,1],[3,5]],[[0,2],[4,2]],[[3,0],[3,1]]],
[[[4,5],[0,2]],[[3,1],[3,1]],[[2,1],[4,1]],[[5,4],[5,2]],[[3,0],[3,0]],[[0,4],[2,5]]],
[[[3,5],[0,4]],[[5,1],[3,4]],[[2,1],[1,0]],[[0,2],[3,3]],[[1,2],[4,4]],[[5,2],[0,5]]],
[[[0,5],[0,5]],[[3,1],[3,1]],[[2,2],[4,4]],[[1,1],[3,3]],[[4,2],[4,2]],[[0,0],[5,5]]],
[[[3,2],[0,1]],[[5,1],[4,4]],[[2,5],[0,4]],[[2,3],[1,4]],[[0,2],[3,3]],[[1,5],[5,0]]],
[[[1,5],[0,3]],[[5,1],[0,1]],[[2,0],[2,5]],[[2,4],[2,4]],[[3,4],[3,4]],[[5,3],[1,0]]],
[[[2,5],[0,4]],[[3,1],[3,1]],[[2,1],[2,1]],[[0,4],[5,4]],[[3,5],[3,0]],[[5,4],[2,0]]],
[[[0,4],[0,1]],[[4,1],[1,3]],[[2,5],[2,3]],[[2,3],[5,4]],[[5,3],[1,0]],[[0,2],[4,5]]],
[[[3,3],[0,4]],[[0,1],[1,4]],[[2,5],[0,3]],[[1,2],[4,2]],[[5,4],[0,5]],[[1,5],[2,3]]],
[[[4,5],[0,3]],[[3,1],[5,3]],[[2,5],[2,1]],[[4,1],[4,2]],[[4,0],[3,2]],[[0,0],[1,5]]],
[[[2,3],[0,4]],[[5,1],[5,1]],[[2,0],[4,0]],[[3,5],[3,5]],[[4,1],[2,1]],[[0,2],[4,3]]],
[[[2,3],[0,4]],[[5,1],[1,2]],[[2,5],[3,3]],[[1,5],[0,0]],[[4,1],[3,0]],[[5,4],[4,2]]],
[[[4,4],[0,3]],[[1,1],[0,3]],[[2,2],[4,0]],[[5,1],[3,2]],[[0,5],[5,3]],[[5,2],[4,1]]],
[[[2,5],[0,4]],[[5,1],[0,5]],[[2,1],[2,5]],[[0,1],[3,4]],[[4,1],[3,2]],[[3,4],[3,0]]],
[[[5,5],[0,0]],[[3,1],[3,1]],[[2,4],[2,4]],[[1,1],[3,3]],[[4,4],[2,2]],[[5,0],[5,0]]],
[[[5,2],[0,2]],[[1,1],[4,4]],[[2,5],[0,4]],[[3,3],[5,0]],[[0,2],[4,5]],[[1,3],[1,3]]],
[[[5,4],[0,4]],[[1,1],[1,1]],[[2,0],[4,0]],[[3,3],[3,3]],[[5,2],[5,4]],[[0,2],[5,2]]],
[[[5,3],[0,5]],[[4,1],[3,5]],[[2,2],[3,0]],[[1,0],[1,3]],[[2,1],[5,4]],[[4,4],[0,2]]],
[[[3,5],[0,2]],[[2,1],[2,4]],[[2,5],[3,4]],[[3,1],[0,3]],[[4,0],[4,1]],[[0,1],[5,5]]],
[[[2,2],[0,4]],[[5,1],[3,1]],[[2,0],[4,0]],[[3,5],[3,1]],[[3,1],[5,5]],[[0,2],[4,4]]],
[[[0,1],[0,4]],[[3,1],[4,3]],[[2,3],[5,4]],[[5,5],[1,0]],[[2,2],[4,1]],[[2,5],[0,3]]],
[[[3,0],[0,2]],[[2,1],[5,4]],[[2,5],[3,5]],[[3,4],[4,2]],[[1,0],[5,4]],[[0,1],[3,1]]],
[[[3,3],[0,5]],[[5,1],[4,0]],[[2,2],[1,1]],[[1,0],[5,4]],[[2,2],[3,3]],[[4,4],[5,0]]],
[[[3,1],[0,3]],[[2,1],[3,4]],[[2,2],[0,3]],[[5,5],[4,1]],[[2,0],[5,4]],[[1,5],[0,4]]],
[[[3,4],[0,0]],[[2,1],[5,1]],[[2,4],[5,2]],[[1,0],[5,3]],[[3,0],[5,4]],[[4,1],[3,2]]],
[[[5,5],[0,2]],[[4,1],[3,1]],[[2,5],[2,0]],[[3,4],[3,1]],[[3,1],[4,4]],[[5,2],[0,0]]],
[[[3,0],[0,5]],[[4,1],[0,2]],[[2,1],[3,0]],[[4,4],[3,5]],[[1,5],[1,3]],[[5,2],[4,2]]],
[[[3,1],[0,0]],[[0,1],[5,1]],[[2,2],[2,2]],[[3,0],[3,5]],[[4,4],[4,4]],[[5,5],[3,1]]],
[[[2,1],[0,2]],[[5,1],[3,4]],[[2,5],[3,4]],[[3,0],[1,3]],[[4,1],[4,0]],[[0,5],[2,5]]],
[[[3,4],[0,1]],[[5,1],[4,0]],[[2,4],[3,4]],[[5,0],[0,2]],[[3,2],[5,3]],[[2,1],[5,1]]],
[[[5,2],[0,2]],[[1,1],[0,5]],[[2,5],[4,3]],[[3,3],[0,5]],[[0,2],[3,4]],[[1,4],[1,4]]],
[[[2,2],[0,4]],[[3,1],[5,1]],[[2,0],[4,0]],[[3,1],[3,5]],[[5,5],[3,1]],[[0,2],[4,4]]],
[[[5,3],[0,5]],[[1,1],[4,3]],[[2,1],[5,4]],[[4,0],[3,4]],[[2,2],[0,3]],[[2,0],[5,1]]],
[[[2,2],[0,3]],[[5,1],[5,4]],[[2,0],[3,4]],[[2,5],[0,4]],[[3,1],[1,4]],[[0,1],[3,5]]],
[[[0,5],[0,3]],[[1,1],[2,4]],[[2,4],[1,4]],[[0,3],[5,3]],[[2,4],[2,1]],[[5,3],[5,0]]],
[[[3,5],[0,0]],[[2,1],[0,4]],[[2,3],[5,5]],[[4,2],[4,3]],[[1,0],[5,4]],[[3,1],[1,2]]],
[[[5,5],[0,4]],[[3,1],[3,4]],[[2,5],[0,3]],[[1,2],[5,3]],[[1,4],[0,0]],[[1,2],[2,4]]],
[[[3,5],[0,3]],[[4,1],[3,1]],[[2,0],[4,4]],[[2,3],[1,2]],[[2,5],[5,4]],[[0,5],[0,1]]],
[[[4,0],[0,5]],[[1,1],[5,4]],[[2,2],[3,4]],[[1,2],[0,3]],[[3,5],[4,3]],[[0,1],[2,5]]],
[[[2,4],[0,4]],[[0,1],[1,5]],[[2,3],[1,2]],[[5,0],[3,0]],[[3,3],[1,4]],[[2,5],[5,4]]],
[[[2,3],[0,0]],[[5,1],[3,0]],[[2,3],[3,1]],[[4,5],[4,1]],[[4,1],[5,2]],[[2,0],[5,4]]],
[[[1,1],[0,5]],[[5,1],[3,0]],[[2,2],[3,3]],[[1,0],[5,3]],[[4,4],[0,5]],[[2,2],[4,4]]],
[[[3,5],[0,0]],[[5,1],[4,1]],[[2,4],[5,2]],[[1,2],[0,3]],[[1,2],[4,0]],[[4,3],[3,5]]],
[[[2,5],[0,4]],[[3,1],[1,0]],[[2,1],[3,4]],[[0,4],[3,5]],[[3,5],[4,5]],[[2,0],[2,1]]],
[[[5,0],[0,1]],[[2,1],[3,4]],[[2,5],[0,4]],[[2,2],[1,0]],[[3,3],[4,5]],[[1,5],[4,3]]],
[[[5,0],[0,4]],[[2,1],[0,4]],[[2,3],[1,0]],[[5,2],[4,2]],[[3,3],[5,4]],[[5,3],[1,1]]],
[[[3,2],[0,1]],[[0,1],[2,5]],[[2,4],[3,2]],[[5,3],[5,4]],[[0,4],[0,5]],[[4,1],[3,1]]],
[[[4,1],[0,1]],[[0,1],[3,4]],[[2,5],[3,5]],[[2,4],[3,2]],[[5,1],[3,0]],[[0,4],[2,5]]],
[[[1,5],[0,2]],[[5,1],[0,1]],[[2,3],[2,3]],[[0,4],[5,4]],[[3,4],[3,4]],[[5,2],[1,0]]],
[[[5,2],[0,2]],[[1,1],[3,4]],[[2,5],[0,0]],[[3,3],[4,1]],[[0,2],[5,5]],[[1,3],[4,4]]],
[[[3,2],[0,3]],[[0,1],[2,1]],[[2,5],[5,0]],[[4,3],[1,5]],[[0,4],[1,5]],[[4,4],[3,2]]],
[[[3,5],[0,4]],[[0,1],[2,0]],[[2,5],[3,4]],[[1,3],[0,3]],[[2,4],[4,1]],[[2,1],[5,5]]],
[[[2,1],[0,4]],[[5,1],[3,1]],[[2,3],[5,3]],[[5,0],[0,2]],[[4,1],[3,0]],[[4,4],[2,5]]],
[[[5,4],[0,5]],[[2,1],[3,5]],[[2,2],[4,3]],[[1,1],[4,0]],[[0,3],[3,4]],[[1,5],[0,2]]],
[[[4,0],[0,2]],[[5,1],[5,0]],[[2,3],[1,2]],[[0,3],[3,5]],[[4,3],[1,1]],[[4,5],[4,2]]]]

superflips

# 3x3x3

In [6]:
from collections import defaultdict

def evaluate3(model, scrambles, number_of_evaluations, her, seed=None):
    env = rubiks.RubiksEnv(3, unsolved_reward=-1.0, seed=seed)
    goal = env.get_observation()
    
    eval_model = torch.load(model)
    eval_model.to(device)
    
    max_tries = 1000
    
    solved = 0
    tries_solved = []
    cycles = 0
    
    for i in range(number_of_evaluations):
        hashes = defaultdict(list)
        state = env.reset(scrambles)
        hashes[hash(state.tostring())] = [0]*env.action_space.n
        done = 0
        tries = 0
        
        while not done and tries < max_tries:
            mask = hashes[hash(state.tostring())]
            if her:
                action = eval_model.act(state, goal, 0)
            else:
                action = eval_model.act(state, 0, mask)
            
            next_state, reward, done, info = env.step(action)
            
            hstate = state.copy()
            state = next_state
            
            tries += 1
            
            h = hash(state.tostring())
            if h in hashes.keys():
                hashes[hash(hstate.tostring())][action] = -999
                cycles += 1
            else:
                hashes[h] = [0]*env.action_space.n
        if done:
            solved += 1
            tries_solved.append(tries)
    
    percentage_solved = solved / number_of_evaluations
    percentage_cycles = cycles / number_of_evaluations
    average_tries = np.average(tries_solved)
    
    return percentage_solved, percentage_cycles, average_tries

models = []
# models.append(('Baseline','./models/Baseline/model.pt', False))
# models.append(('Mixed','./models/Mixed/model.pt', False))
# models.append(('Joe','./models/Joe/model.pt', False))
# models.append(('Sutskever','./models/Sutskever/model.pt', False))
# models.append(('Prioritised0.5','./models/Prioritised0.5/model.pt', False))
# models.append(('Prioritised0.7','./models/Prioritised0.7/model.pt', False))
# models.append(('Prioritised0.9','./models/Prioritised0.9/model.pt', False))
# models.append(('Dueling','./models/Dueling/model.pt', False))
models.append(('Double','./models/Double3/model.pt', False))
# models.append(('Linear All','./models/Linear_all/model.pt', False))
# models.append(('Elu','./models/Elu/model.pt', False))
# models.append(('Leaky Relu','./models/Leaky_relu3/model.pt', False))
# models.append(('Baseline HER','./models/Baseline_HER/model.pt', True))
# models.append(('Mixed HER','./models/Mixed_HER/model.pt', True))
# models.append(('Joe HER','./models/Joe_HER/model.pt', True))
# models.append(('Sutskever HER','./models/Sutskever_HER/model.pt', True))
# models.append(('Prioritised0.5 HER','./models/Prioritised0.5_HER/model.pt', True))
# models.append(('Prioritised0.7 HER','./models/Prioritised0.7_HER/model.pt', True))
# models.append(('Prioritised0.9 HER','./models/Prioritised0.9_HER/model.pt', True))
# models.append(('Dueling HER','./models/Dueling_HER/model.pt', True))
# models.append(('Double HER','./models/Double_HER/model.pt', True))
# models.append(('Linear All HER','./models/Linear_all_HER/model.pt', True))
# models.append(('Elu HER','./models/Elu_HER/model.pt', True))

for model in models:
    print(model[0])
   
    percentages_solved = []
    percentages_cycles = []
    averages_tries = []
    for scrambles in range(30):
        percentage_solved, percentage_cycles, average_tries = evaluate3(model[1], scrambles + 1, 640, model[2], seed=42)
        print('Scrambles: ', scrambles + 1,
              'Percentage Solved: ', percentage_solved,
              'Percentage Cycles: ', percentage_cycles,
              'Average Tries: ', average_tries)
        percentages_solved.append(percentage_solved)
        percentages_cycles.append(percentage_cycles)
        averages_tries.append(average_tries)
    
    total_points = 0
    total_idx = 0
    for i, p in enumerate(percentages_solved):
        total_idx += i+1
        total_points += averages_tries[i]*p
    
    print('Normalised Score: ', total_points/total_idx)
        
    percentage_solved, percentage_cycles, average_tries = evaluate3(model[1], 1000, 640, model[2], seed=42)
    print('Scrambles: ', 1000, 
          'Percentage Solved: ', percentage_solved,
          'Percentage Cycles: ', percentage_cycles,
          'Average Tries: ', average_tries)
        
    plt.figure()
    plt.plot([scrambles + 1 for scrambles in range(30)], percentages_solved)
    plt.plot([scrambles + 1 for scrambles in range(30)], percentages_cycles)
    plt.xlabel('Scramble Distance')
    plt.ylabel('Percentage')
    plt.legend(('Percentage Solved', 'Percentage Cycles'))
    plt.show()
    
    plt.figure()
    plt.plot([scrambles + 1 for scrambles in range(30)], averages_tries)
    plt.xlabel('Scramble Distance')
    plt.ylabel('Average Number of Tries')
    plt.show()

Double
Scrambles:  1 Percentage Solved:  1.0 Percentage Cycles:  0.0 Average Tries:  1.0
Scrambles:  2 Percentage Solved:  1.0 Percentage Cycles:  0.0 Average Tries:  2.0
Scrambles:  3 Percentage Solved:  1.0 Percentage Cycles:  0.0 Average Tries:  2.64375
Scrambles:  4 Percentage Solved:  1.0 Percentage Cycles:  0.0 Average Tries:  3.459375
Scrambles:  5 Percentage Solved:  0.990625 Percentage Cycles:  9.3546875 Average Tries:  4.220820189274448
Scrambles:  6 Percentage Solved:  0.9578125 Percentage Cycles:  42.065625 Average Tries:  5.024469820554649
Scrambles:  7 Percentage Solved:  0.8640625 Percentage Cycles:  135.359375 Average Tries:  5.705244122965642
Scrambles:  8 Percentage Solved:  0.7125 Percentage Cycles:  286.0921875 Average Tries:  6.201754385964913
Scrambles:  9 Percentage Solved:  0.6296875 Percentage Cycles:  368.334375 Average Tries:  6.746898263027295
Scrambles:  10 Percentage Solved:  0.44375 Percentage Cycles:  553.0671875 Average Tries:  7.147887323943662
Scrambl

RuntimeError: CUDA error: unspecified launch failure

defaultdict(list, {'2': [1, 1, 1, 1, 1]})